In [1]:
import os,sys
import yaml
from Cracked_Detection.utils.main_utils import read_yaml_file
from Cracked_Detection.logger import logging
from Cracked_Detection.exception import AppException
from Cracked_Detection.entity.config_entity import ModelTrainerConfig
from Cracked_Detection.entity.artifacts_entity import ModelTrainerArtifact

In [4]:
os.system("unzip data.zip")

1

In [3]:
os.system("rm data.zip")

1

In [5]:
class ModelTrainer:
    def __init__(
        self,
        model_trainer_config: ModelTrainerConfig,
    ):
        self.model_trainer_config = model_trainer_config


    

    def initiate_model_trainer(self,) -> ModelTrainerArtifact:
        logging.info("Entered initiate_model_trainer method of ModelTrainer class")

        try:
            logging.info("Unzipping data")
            os.system("unzip data.zip")
            os.system("rm data.zip")

            with open("data.yaml", 'r') as stream:
                num_classes = str(yaml.safe_load(stream)['nc'])

            model_config_file_name = self.model_trainer_config.weight_name.split(".")[0]
            print(model_config_file_name)

            config = read_yaml_file(f"artifacts/data_ingestion/feature_store/{model_config_file_name}.yaml")

            config['nc'] = int(num_classes)


            with open(f'yolov8/models/custom_{model_config_file_name}.yaml', 'w') as f:
                yaml.dump(config, f)

            os.system(f"cd yolov8/ && python train.py --img 416 --batch {self.model_trainer_config.batch_size} --epochs {self.model_trainer_config.no_epochs} --data ../data.yaml --cfg ./models/custom_yolov8s.yaml --weights {self.model_trainer_config.weight_name} --name yolov8s_results  --cache")
            os.system("cp yolov5/runs/train/yolov8s_results/weights/best.pt yolov8/")
            os.makedirs(self.model_trainer_config.model_trainer_dir, exist_ok=True)
            os.system(f"cp yolov5/runs/train/yolov5s_results/weights/best.pt {self.model_trainer_config.model_trainer_dir}/")
           
            os.system("rm -rf yolov8/runs")
            os.system("rm -rf train")
            os.system("rm -rf valid")
            os.system("rm -rf data.yaml")

            model_trainer_artifact = ModelTrainerArtifact(
                trained_model_file_path="yolov8/best.pt",
            )

            logging.info("Exited initiate_model_trainer method of ModelTrainer class")
            logging.info(f"Model trainer artifact: {model_trainer_artifact}")

            return model_trainer_artifact


        except Exception as e:
            raise AppException(e, sys)

            with open(f'yolov8/models/custom_{model_config_file_name}.yaml', 'w') as f:
                yaml.dump(config, f)

            os.system(f"cd yolov8/ && python train.py --img 416 --batch {self.model_trainer_config.batch_size} --epochs {self.model_trainer_config.no_epochs} --data ../data.yaml --cfg ./models/custom_yolov8s.yaml --weights {self.model_trainer_config.weight_name} --name yolov8s_results  --cache")
            os.system("cp yolov5/runs/train/yolov8s_results/weights/best.pt yolov8/")
            os.makedirs(self.model_trainer_config.model_trainer_dir, exist_ok=True)
            os.system(f"cp yolov5/runs/train/yolov5s_results/weights/best.pt {self.model_trainer_config.model_trainer_dir}/")
           
            os.system("rm -rf yolov8/runs")
            os.system("rm -rf train")
            os.system("rm -rf valid")
            os.system("rm -rf data.yaml")

In [1]:
from flask import Flask, request, jsonify, render_template
from Cracked_Detection.pipeline.training_pipeline import TrainPipeline
from Cracked_Detection.utils.main_utils import decodeImage, encodeImageIntoBase64
from ultralytics import YOLO  
import base64
import os,glob,time,re
import shutil
from flask_cors import CORS, cross_origin

In [8]:
os.chdir("../")

In [9]:
pwd

'd:\\CrackedSegmentationModel'

In [11]:
filename = "data/inputImage.jpg"

In [12]:
filename 

'data/inputImage.jpg'

In [24]:
parent_directory = "runs/segment/"
parent_directory

'runs/segment/'

In [25]:
directories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]

In [26]:
directories

['predict',
 'predict10',
 'predict11',
 'predict2',
 'predict3',
 'predict4',
 'predict5',
 'predict6',
 'predict7',
 'predict8',
 'predict9',
 'train',
 'train2']

In [27]:
pattern = "predict"

In [28]:
filtered_directories = [d for d in directories if d.startswith(pattern) and d[len(pattern):].isdigit()]
filtered_directories

['predict10',
 'predict11',
 'predict2',
 'predict3',
 'predict4',
 'predict5',
 'predict6',
 'predict7',
 'predict8',
 'predict9']

In [29]:
latest_directory = max(filtered_directories)
latest_directory

'predict9'

In [30]:
latest_directory = max(filtered_directories, key=lambda x: int(re.search(r'\d+$', x).group()))
latest_directory

'predict11'

In [21]:
image_path = os.path.join(parent_directory, latest_directory, "inputImage.jpg").replace('\\', '/')
image_path

'runs/segment/predict7/inputImage.jpg'